# Load data


In [17]:
import pandas as pd
import numpy as np
import zipfile
from tqdm import tqdm
import re
import json

In [2]:
names_def = pd.read_csv('../data/names_def_2015_2019.csv')

In [2]:
import json
import os

# Initialize an empty list to store the data
data_list = []

# Open the ZIP file
with zipfile.ZipFile('/Users/bkohler/polybox/news_wisconsin/output_wisconsin.zip', 'r') as z:
    # Iterate over each file in the ZIP archive
    for filename in tqdm(z.namelist()):
        # Check if the file is a JSON file
        if filename.endswith('.json'):
            # Read the JSON file
            with z.open(filename) as f:
                data = json.load(f)
                data_list.extend(data["articles"])



100%|██████████| 302/302 [00:16<00:00, 18.54it/s]


In [3]:
len(data_list)

1071988

In [7]:
[js for js in data_list if 'afcf48e58453d0d26a3598a3e92c10e6' in js['_id']]

[{'title': 'Prosecutors say Newport News police were justified in killing man after 2019 ambush',
  'author': 'Peter Dujardin',
  'published_date': '2020-12-24 12:37:00',
  'published_date_precision': 'full',
  'link': 'http://www.dailypress.com/news/crime/dp-nw-fz20-police-shooting-20201224-nrrpg3wm7bgpjnhyws5i2cqhaa-story.html',
  'clean_url': 'dailypress.com',
  'excerpt': 'Civilian on a police "ride-along program" was shot several times by the suspect; police officer was also shot.',
  'summary': "'Based on these facts, it is clear that Jerome Uzzle posed an imminent threat of serious physical harm to members of the public as well as to officers in his vicinity,' the report said. 'Therefore, Officer Kidder's use of deadly force in the death of Mr. Uzzle was justified by the facts and the law. This office will take no further action in this matter.'",
  'rights': 'dailypress.com',
  'rank': 6128,
  'topic': 'news',
  'country': 'US',
  'language': 'en',
  'authors': 'Peter Dujardin'

In [5]:
# Convert the list of dictionaries to a DataFrame
news_df = pd.DataFrame(data_list)

In [42]:
any([not isinstance(v,str) for v in names_def.last_name.tolist()])

last_names = [re.escape(v).lower().capitalize() for v in names_def.last_name.tolist() if isinstance(v,str)]

In [45]:
def get_last_names_in_text(last_names, text):
    last_names_found = []
    for last_name in last_names:
        if last_name in text:
            last_names_found.append(last_name)
    return last_names_found


In [50]:
def get_last_names_in_text(last_names, text):
    # Create a regex pattern to match any of the last names
    pattern = re.compile("|".join(last_names))
    
    # Find all matches in the text
    last_names_found = pattern.findall(text)
    
    return last_names_found

In [52]:
news_df['summary'][100]

"'The weather was good and there was no flash flood,' said Deden Ridwansyah, chief of the Bandung Search and Rescue Office. 'Those children who drowned were holding each others' hands. One of them slipped and the others followed,' Ridwansyah said."

In [60]:
"Plaza" in last_names

True

In [62]:
names_def.last_name.str.contains("Plaza")

0         False
1         False
2         False
3         False
4         False
          ...  
612719    False
612720    False
612721    False
612722    False
612723    False
Name: last_name, Length: 612724, dtype: object

In [53]:
all_last_names_found = []
for text in tqdm(news_df['summary'][:100]):
    last_names_found = get_last_names_in_text(last_names, text)
    all_last_names_found.append(last_names_found)


100%|██████████| 100/100 [00:08<00:00, 11.31it/s]


Problem here: last names resemble regular words like "Friday", try full names instead

In [76]:
full_names = names_def.first_name.str.lower().str.capitalize() + " " + names_def.last_name.str.lower().str.capitalize()
full_names = [re.escape(v) for v in full_names if isinstance(v,str)]

In [80]:
all_full_names_found = []
for text in tqdm(news_df['summary'][:1000]):
    full_names_found = get_last_names_in_text(full_names, text)
    all_full_names_found.append(full_names_found)


100%|██████████| 1000/1000 [03:04<00:00,  5.43it/s]


In [86]:
news_df['summary'][3]

"They may be 'allies,' but they certainly don't agree on everything.Democratic mayoral contender Eric Adams signaled a significant gap between he and Mayor de Blasio's thinking when it comes to COVID vaccine mandates Friday when Adams said he'd mandate that school children receive shots in order to attend class.'This is a city and country where we do vaccinate. I was vaccinated for smallpox, for mumps, measles and so many others,' said Adams during an interview on WCBS 880. 'We already have a system in place that states before you start school you receive your vaccination. It is to protect the child and the student population.'Jesah Hernandez, a student at Lehman High School, gets the Pfizer COVID-19 vaccine in Bronx, New York. Jesah Hernandez, a student at Lehman High School, gets the Pfizer COVID-19 vaccine in Bronx, New York. (Mark Lennihan/)Adams, who's facing longshot Republican candidate in this November's election, said as long as a COVID vaccine for kids has been approved by th

In [88]:
count_tokens


NameError: name 'count_tokens' is not defined

In [93]:
tokens = 0
for text in tqdm(news_df['summary']):
    if isinstance(text,str):
        tokens += len(text.split())

100%|██████████| 1071988/1071988 [00:19<00:00, 53901.65it/s]


In [107]:
500 * 0.075

37.5

In [98]:
news_df['summary'].il

"Overall, tax collections were $1.8 billion higher than forecast by the state Division of the Budget in its first quarter update, released last month. Tax receipts through the end of September were $13.4 billion higher than they were through the same period last year when the COVID crisis shuttered businesses and the state's economy ground to a halt."

In [111]:
news_df.to_csv('../news_wisconsin.csv', index=False)

In [87]:
count_tokens = news_df['summary'].str.split().apply(len)

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x10510d310>>
Traceback (most recent call last):
  File "/Applications/miniconda3/envs/newscatchervenv/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


TypeError: object of type 'NoneType' has no len()

In [81]:
all_full_names_found

[[],
 ['Daniel Jones',
  'Daniel Jones',
  'Andrew Thomas',
  'Aaron Donald',
  'Justin Hill'],
 [],
 ['Eric Adams', 'Eric Adams', 'Eric Adams'],
 [],
 [],
 [],
 [],
 [],
 [],
 ['David Miller'],
 [],
 [],
 [],
 [],
 [],
 [],
 ['Daniel Jones',
  'Daniel Jones',
  'Daniel Jones',
  'Andrew Thomas',
  'Justin Hill'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['Matthew Hoppe'],
 ['Antonio Brown'],
 [],
 ['Patrick Lynch', 'Patrick Lynch'],
 ['Michael Kennedy'],
 [],
 ['Richard Drew', 'Patrick Lynch'],
 [],
 [],
 [],
 [],
 [],
 [],
 ['Aaron Hicks'],
 ['Karen Harris', 'Karen Harris', 'Karen Harris', 'Karen Harris'],
 [],
 [],
 [],
 ['Andres Rivera'],
 ['Eric Adams',
  'Eric Adams',
  'Bernard Adams',
  'Eric Adams',
  'Eric Adams',
  'Bernard Adams',
  'Michael Grey',
  'Eric Adams',
  'Eric Adams'],
 ['Jeffrey Wright', 'Elijah Wright'],
 [],
 ['Andrew Thomas', 'Daniel Jones', 'Blake Martin', 'Leonard Williams'],
 [],
 ['Kyle Rittenhouse'],
 ['David Hand',
  'Davi

In [46]:
# news_df.summary.str.contains("|".join(last_names))

# Sample of crime news

In [114]:
news_df_sample = news_df.sample(100000)

In [124]:
news_df_sample.reset_index(inplace=True)

In [143]:
crime_words = ["crime", "criminal", "verdict","arrest","prosecution"]

In [144]:
news_df_sample["is_crime"] = news_df_sample["summary"].str.contains("|".join(crime_words))

In [145]:
news_df_sample["is_crime"].sum()

np.int64(10390)

In [133]:
news_df_sample.dropna(subset=["summary"], inplace=True)

In [222]:
print(news_df_sample[news_df_sample.is_crime].iloc[49].summary)


Extremism came clearly into focus this week.The Department of Homeland Security issued a warning that the US was in a 'heightened threat environment' over the next six months. Reasons included the midterm elections as well as the immigration crisis at the border with Mexico.The warning also noted the looming Supreme Court decision on abortion, a concern that became a reality Wednesday when a man was arrested in connection to an assassination attempt on Supreme Court Justice Brett Kavanaugh.Stories of extremism did not stop there. On Thursday night in front of a national television audience, the House panel investigating the Jan. 6, 2021, insurrection included dramatic capitol police dispatch audio audio and details from panel members that placed the blame on former President Donald Trump.Trump downplayed the investigation on social media and House Republicans believe the findings are a distraction from current problems that will backfire on Democrats during the midterms. Earlier in the

In [223]:
manual_classification_correct=[1,0,0,0,1,1,0,1,0,1,1,0,0,0,1,1,1,1,0,0,0,1,0,1,1,1,1,1,0,0,1,1,1,1,0,1,1,0,1,0,0,0,1,1,1,1,0,1,0,0]

In [224]:
manual_classification_correct=[True if v==1 else False for v in manual_classification_correct]

In [226]:
sum(manual_classification_correct)

28

In [227]:
crime_related_articles_samples = news_df_sample[news_df_sample.is_crime].iloc[0:len(manual_classification_correct)].iloc[manual_classification_correct][["summary","index","is_crime"]]

In [228]:
random_sample_non_crime = news_df_sample[~news_df_sample.is_crime].sample(len(crime_related_articles_samples)*3)[["summary","index","is_crime"]]

In [231]:
test_sample = pd.concat([crime_related_articles_samples, random_sample_non_crime])

In [232]:
test_sample.to_csv('../data/test_sample_crime.csv', index=False)

In [234]:
test_sample.shape

(112, 3)

In [237]:
test_sample.iloc[0:5].summary

14     A 'person of interest' has been named by polic...
62     Officials are asking the public for help ident...
65     EAU CLAIRE — Three of six men accused of illeg...
79     A suspect already facing an attempted murder c...
117    Woman held at gunpoint as her car gets stolen ...
Name: summary, dtype: object

In [6]:
news_df[news_df.index==373789]

,title,author,published_date,published_date_precision,link,clean_url,excerpt,summary,rights,rank,topic,country,language,authors,media,is_opinion,twitter_account,_score,_id
373789,Prosecutors say Newport News police were justi...,Peter Dujardin,2020-12-24 12:37:00,full,http://www.dailypress.com/news/crime/dp-nw-fz2...,dailypress.com,"Civilian on a police ""ride-along program"" was ...","'Based on these facts, it is clear that Jerome...",dailypress.com,6128,news,US,en,Peter Dujardin,https://www.dailypress.com/resizer/S-n-tIN4WE6...,False,None,1.841846,afcf48e58453d0d26a3598a3e92c10e6


# Batch api sample

In [14]:
news_df[news_df.published_date.str.startswith("2019")].shape

(118652, 19)

In [15]:
news_df_sample_20k = news_df[news_df.published_date.str.startswith("2019")].sample(20000, random_state=42)

In [18]:
# save json as binary

news_df_sample_20k.to_json('../data/news_wisconsin_sample_20k.json', orient='records', lines=True)


In [20]:
with open('../data/news_wisconsin_sample_20k.json', 'rb') as f:
    data = f.readlines()

In [23]:
data[0][]

TypeError: byte indices must be integers or slices, not str

In [16]:
news_df_sample_20k.reset_index(inplace=True)

news_df_sample_20k.to_csv('../data/news_wisconsin_sample_20k.csv', index=False)